In [47]:
import pandas as pd
from py2neo import Graph, Node
from tqdm import tqdm

In [48]:
df = pd.read_csv('data_test.csv', delimiter=';', encoding='utf-8')

In [49]:
df.head()

,id события,ФИО участника события 1,ФИО участника события 2
0,189,Галчевская Карина Владимировна,Белоновская Анастасия Семеновна
1,206,Офицеров Олег Романович,Сапожник Борис Валерьевич
2,445,Жандарова Лариса Германовна,Чемодуров Дамир Русланович
3,503,Масимова Яна Дамировна,Мингажетдинов Рамиль Семенович
4,571,Мухтарова Алена Яковлевна,Щербатенко Ольга Робертовна


Подключаемся к графовой базе данных с помощью py2neo

In [50]:
graph = Graph("bolt://localhost:7687", auth=("neo4j", "0123456789"))

Очищаем графовую базу данных

In [51]:
graph.delete_all()

In [52]:
# Создаем узлы
nodes = set(df['ФИО участника события 1']).union(set(df['ФИО участника события 2']))

for node in tqdm(nodes):
    graph.create(Node("Person", name=node))

100%|██████████| 9899/9899 [00:56<00:00, 176.15it/s]


In [53]:
# Создаем связи
query = """
MATCH (p1:Person {name: $name1})
MATCH (p2:Person {name: $name2})
MERGE (p1)-[r:PARTICIPATED_IN {event_id: $event_id}]->(p2)
"""

for index, row in tqdm(df.iterrows(), total=len(df)):
    graph.run(query, name1=row['ФИО участника события 1'], name2=row['ФИО участника события 2'],
              event_id=row['id события'])


100%|██████████| 5000/5000 [01:18<00:00, 63.55it/s]


In [54]:
query = """
MATCH (p1:Person)-[r:PARTICIPATED_IN]->(p2:Person)
RETURN p1.name AS Person1, p2.name AS Person2, r.event_id AS Event
"""

result = graph.run(query)

data = pd.DataFrame(result)

data.head()

,0,1,2
0,Зелинский Геннадий Артурович,Зазорин Вадим Аркадьевич,177407
1,Угрюмова Марина Валентиновна,Фондорина Ксения Викторовна,305692
2,Семиошкина Алиса Владимировна,Рассадкин Леонид Федорович,190143
3,Шалашова Алена Федоровна,Шинкарева Яна Георгиевна,131201
4,Фурсаева Лидия Дмитриевна,Пересыпкина Маргарита Ефимовна,112526


In [55]:
query = """
MATCH ()-[r]->()
RETURN DISTINCT type(r), case when startNode(r) <> endNode(r) then "directed" else "undirected" end as direction
"""

result = graph.run(query).to_data_frame()

if len(result) == 1 and result.iloc[0]['direction'] == 'undirected':
    print("Граф не является направленным")
else:
    print("Граф является направленным")


Граф является направленным


## Обнаружение сообществ
Алгоритмы обнаружения сообществ позволяют выделить группы узлов, которые имеют сильные внутренние связи, но слабые связи с узлами в других группах. Например, мы можем использовать алгоритм Лувена для обнаружения сообществ в нашем графе.

In [59]:
import networkx as nx

# Загрузка данных из Neo4j в объект networkx.Graph
query = """
MATCH (p1:Person)-[r:PARTICIPATED_IN]->(p2:Person)
RETURN p1.name as source, p2.name as target, count(r) as weight
"""

data = graph.run(query).to_data_frame()
G = nx.from_pandas_edgelist(data, source='source', target='target', edge_attr='weight')

# Обнаружение сообществ с помощью алгоритма Лувейна
partition = nx.algorithms.community.label_propagation.label_propagation_communities(G)

# Вывод количества сообществ
print(f"Количество сообществ: {len(partition)}")

Количество сообществ: 4909


## Центральность вершин
Алгоритмы центральности вершин позволяют определить наиболее важные узлы в графе, которые играют ключевую роль в связях между другими узлами. Например, мы можем использовать алгоритм центральности посредничества (betweenness centrality) для выявления наиболее важных узлов в нашем графе.

In [71]:
import networkx as nx

# Создание графа на основе Neo4j
G = nx.Graph()
for rel in tqdm(graph.match()):
    G.add_edge(rel.start_node.identity, rel.end_node.identity)

# Вычисление центральности посредничества
betweenness_centrality = nx.betweenness_centrality(G)

# Вывод наиболее важных узлов
top_nodes = sorted(betweenness_centrality, key=betweenness_centrality.get, reverse=True)[:10]
for node in top_nodes:
    print(f"{node}: {betweenness_centrality[node]}")


100%|██████████| 5000/5000 [00:48<00:00, 102.36it/s]


1850: 2.501007906186193e-05
1669: 1.8578915874526003e-06
2648: 8.370720339072156e-07
3069: 7.758228606944924e-07
6162: 7.758228606944924e-07
1756: 7.554064696235848e-07
1166: 7.349900785526771e-07
3198: 7.145736874817694e-07
5971: 7.145736874817694e-07
4782: 3.062458660636155e-07


В результате было выведено 10 наиболее важных узлов (по убыванию значения центральности).

Значения центральности могут интерпретироваться как мера того, насколько важными являются узлы для связности графа. Чем выше значение центральности, тем более важным является узел.

В данном случае, узел с идентификатором 1850 имеет наивысшее значение центральности. Это означает, что этот узел наиболее важен для связности графа, и вероятно, он является ключевым игроком в в той сети, которую мы анализируем. Остальные узлы в списке также имеют высокие значения центральности, что может указывать на их значимость для связности графа.

Простыми словами, данная мера показывает нам насколько часто рассматриваемая вершина является «перевалочным пунктом» при переходах от одной вершины графа до любой другой. Она позволяет достаточно хорошо определять «узкие места» в графе — вершины, входящие в состав ребра или набора ребер, соединяющих два ярко выраженных кластера.

In [77]:
query = """
MATCH (n) WHERE id(n) = 1850 RETURN n
"""

result = graph.run(query).to_series()

print(result)

0    {'name': 'Ахромеева Алина Ивановна'}
dtype: object
